# k-Nearest Neighbors (MADlib v1.10+)
Finds k nearest data points to a given data point and outputs majority vote value of output classes in case of classification, and average value of target values in case of regression.

In [8]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [9]:
%sql postgresql://gpdbchina@10.194.10.68:55000/madlib
#%sql postgresql://fmcquillan@localhost:5432/madlib
#%sql postgresql://gpadmin@54.197.30.46:10432/gpadmin

u'Connected: gpdbchina@madlib'

In [18]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.10.0-dev, git revision: rel/v1.9.1-47-g2d5a5ed, cmake configuration time: Tue Feb 7 19:45:19 UTC 2017, build type: Release, build system: Linux-2.6.18-238.27.1.el5.hotfix.bz516490, C compiler: gcc 4.4.0, C++ compiler: g++ 4.4.0"


# 1.  Misc testing

In [19]:
%%sql 
DROP TABLE IF EXISTS knn_train_data;
CREATE TABLE knn_train_data (
                    id integer, 
                    data integer[], 
                    label float
                    );

INSERT INTO knn_train_data VALUES
(1, '{1,1}', 1.0),
(2, '{2,2}', 1.0),
(3, '{3,3}', 1.0),
(4, '{4,4}', 1.0),
(5, '{4,5}', 1.0),
(6, '{20,50}', 0.0),
(7, '{10,31}', 0.0),
(8, '{81,13}', 0.0),
(9, '{1,111}', 0.0);

SELECT * from knn_train_data ORDER BY id;

Done.
Done.
9 rows affected.
9 rows affected.


id,data,label
1,"[1, 1]",1.0
2,"[2, 2]",1.0
3,"[3, 3]",1.0
4,"[4, 4]",1.0
5,"[4, 5]",1.0
6,"[20, 50]",0.0
7,"[10, 31]",0.0
8,"[81, 13]",0.0
9,"[1, 111]",0.0


In [23]:
%%sql 
DROP TABLE IF EXISTS knn_test_data;
CREATE TABLE knn_test_data (
                    id integer, 
                    data integer[]
                    );

INSERT INTO knn_test_data VALUES
(1, '{2,1}'),
(2, '{2,6}'),
(3, '{15,40}'),
(4, '{12,1}'),
(5, '{2,90}'),
(6, '{50,45}');

SELECT * from knn_test_data ORDER BY id;

Done.
Done.
6 rows affected.
6 rows affected.


id,data
1,"[2, 1]"
2,"[2, 6]"
3,"[15, 40]"
4,"[12, 1]"
5,"[2, 90]"
6,"[50, 45]"


In [24]:
%%sql
DROP TABLE IF EXISTS madlib_knn_result_classification;
SELECT * FROM madlib.knn( 
                'knn_train_data',      -- Table of training data
                'data',                -- Col name of training data
                'label',               -- Training labels
                'knn_test_data',       -- Table of test data
                'data',                -- Col name of test data
                'id',                  -- Col name of id in test data 
                'madlib_knn_result_classification',  -- Output table
                'c',                   -- Classification
                 3                     -- Number of nearest neighbours
                );
SELECT * from madlib_knn_result_classification ORDER BY id;

Done.
1 rows affected.
6 rows affected.


id,data,prediction
1,"[2, 1]",1.0
2,"[2, 6]",1.0
3,"[15, 40]",0.0
4,"[12, 1]",1.0
5,"[2, 90]",0.0
6,"[50, 45]",0.0


In [25]:
%%sql
DROP TABLE IF EXISTS madlib_knn_result_regression;
SELECT * FROM madlib.knn( 
                'knn_train_data',      -- Table of training data
                'data',                -- Col name of training data
                'label',               -- Training labels
                'knn_test_data',       -- Table of test data
                'data',                -- Col name of test data
                'id',                  -- Col name of id in test data 
                'madlib_knn_result_regression',  -- Output table
                'r',                   -- Regressions
                 3                     -- Number of nearest neighbours
                );
SELECT * from madlib_knn_result_regression ORDER BY id;

Done.
1 rows affected.
6 rows affected.


id,data,prediction
1,"[2, 1]",1.0
2,"[2, 6]",1.0
3,"[15, 40]",0.333333333333
4,"[12, 1]",1.0
5,"[2, 90]",0.0
6,"[50, 45]",0.0
